# 数据分析

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('data/input/train.csv')
test = pd.read_csv('data/input/test_public.csv')
sample = pd.read_csv('data/input/submit_example.csv')

In [3]:
train.head()

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高


In [4]:
test.head()

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9947 entries, 0 to 9946
Data columns (total 5 columns):
content_id         9947 non-null object
content            9947 non-null object
subject            9947 non-null object
sentiment_value    9947 non-null int64
sentiment_word     2169 non-null object
dtypes: int64(1), object(4)
memory usage: 388.6+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2364 entries, 0 to 2363
Data columns (total 2 columns):
content_id    2364 non-null object
content       2364 non-null object
dtypes: object(2)
memory usage: 37.0+ KB


In [7]:
sample.head()

,content_id,subject,sentiment_value,sentiment_word
0,XuPwKCnA2fqNh5vm,价格,0,NaN
1,2jNbDn85goX3IuPE,价格,0,NaN
2,hLgEADQ8sUnvGFK9,价格,0,NaN
3,nZmM7LQsfr03wUaz,价格,0,NaN
4,pwd8MnrthDqLZafe,价格,0,NaN


In [8]:
combine = [train,test]

In [9]:
train.describe()

,sentiment_value
count,9947.000000
mean,0.005429
std,0.574765
min,-1.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [10]:
train.set_index(["content_id","subject"]).count(level="subject")

,content,sentiment_value,sentiment_word
subject,,,
价格,1273,1273,193
内饰,536,536,187
动力,2732,2732,552
外观,489,489,164
安全性,573,573,53
操控,1036,1036,245
油耗,1082,1082,191
空间,442,442,146
舒适性,931,931,269


In [11]:
train.set_index(["content_id","sentiment_value"]).count(level="sentiment_value")

,content,subject,sentiment_word
sentiment_value,,,
-1,1616,1616,952
0,6661,6661,252
1,1670,1670,965


可以看到6661条为中立情感，大概各自有1600条的积极和负面评价

# 分词

In [12]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
train["content_cutted"] = train.content.apply(chinese_word_cut)
test["content_cutted"] = test.content.apply(chinese_word_cut)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf_train = tf_vectorizer.fit_transform(train.content_cutted)
tf_test = tf_vectorizer.fit_transform(test.content_cutted)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/lq/4cxvdwpx3xvbh36yldhr1cjw0000gn/T/jieba.cache
Loading model cost 0.933 seconds.
Prefix dict has been built succesfully.


In [13]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

stop_words_file = "data/stopwordsHIT.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

In [17]:
train_term_matrix = pd.DataFrame(vect.fit_transform(train["content_cutted"]).toarray(), columns=vect.get_feature_names())

In [18]:
train_term_matrix.head()

,000,01,020,03,030,04,040,05,06,07,...,齐活,齿轮,齿轮油,龌龊,龙华,龙潭,龙膜,龙蟠,龙鼎,龟牌
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
train_term_matrix.shape

(9947, 15479)

In [21]:
test_term_matrix = pd.DataFrame(vect.fit_transform(test["content_cutted"]).toarray(), columns=vect.get_feature_names())

In [22]:
test_term_matrix.shape

(2364, 7459)

添加停用词

In [23]:
vect = CountVectorizer(stop_words=frozenset(stopwords))
train_term_matrix = pd.DataFrame(vect.fit_transform(train["content_cutted"]).toarray(), columns=vect.get_feature_names())
test_term_matrix = pd.DataFrame(vect.fit_transform(test["content_cutted"]).toarray(), columns=vect.get_feature_names())

/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [28]:
train_term_matrix.shape

(9947, 5350)

In [27]:
test_term_matrix.shape

(2364, 1666)

去掉常见词和稀疏词

In [26]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。
vect = CountVectorizer(max_df = max_df,
                       min_df = min_df,
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                       stop_words=frozenset(stopwords))
train_term_matrix = pd.DataFrame(vect.fit_transform(train["content_cutted"]).toarray(), columns=vect.get_feature_names())
test_term_matrix = pd.DataFrame(vect.fit_transform(test["content_cutted"]).toarray(), columns=vect.get_feature_names())

/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


# 训练模型

In [29]:
train_term_matrix.head()

,a180,a4,a4l,a6,a6l,a8,abs,ac,acc,act,...,黑内,黑屏,黑桶,黑色,默认,鼻子,齐亮,齐全,齿轮,齿轮油
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


预测标签转化为0，1，2

In [30]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline

lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.subject.values)

In [31]:
y

array([0, 0, 0, ..., 1, 9, 2])

删除content_id content

In [41]:
# train = train.drop(['content_id'], axis=1)
# train = train.drop(['content'], axis=1)
test = test.drop(['content_id'], axis=1)
test = test.drop(['content'], axis=1)

划分验证集

In [35]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(train.content_cutted.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [36]:
print (xtrain.shape)
print (xvalid.shape)

(8952,)
(995,)


In [42]:
from sklearn.linear_model import LogisticRegression

# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain, ytrain)
predictions = clf.predict_proba(xvalid)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: '车载 地图 好像 和 音频 打架 ， 一报 导航 ， 音频 声音 就 小 多 了 ， 我 听 评书 不能 中断 。 你们 都 咋 解决 这个 问题 的 ？'